In [43]:
# load packages
import pandas as pd
import geopandas as gpd
import unidecode 

In [44]:
pois_file_name = './caar_raw.geojson'
pois_df = gpd.read_file(pois_file_name)
pois_df.count()

Name           153
localite       153
code_agence    153
succursale     153
address        153
phone          153
fax            153
position       153
geometry       153
dtype: int64

In [45]:
pois_df.head()

,Name,localite,code_agence,succursale,address,phone,fax,position,geometry
0,01-Adrar,Adrar,408,Oran (400),"N° 03, Centre Commercial ""AMRAD"" - Adrar",049 36 78 83 / 049 36 78 85,049 36 78 85,"27.870513098041783, -0.2878241762255456",POINT Z (-0.28782 27.87051 0.00000)
1,02-Chlef,Chlef,607,Bouzaréah (600),"Rue Ibn Rochd, BP. 137 - Chlef",027 77 15 73,027 77 81 58,"36.170123, 1.334741",POINT Z (1.33474 36.17012 0.00000)
2,02-Chlef,Chlef,732,Bouzaréah (600),"Cite des fonctionnaires, Bt. 1 N°06 - Chlef",027 79 04 20,027 79 04 20,"36.169331, 1.341806",POINT Z (1.34181 36.16933 0.00000)
3,03-Laghouat,Laghouat,621,Bouzaréah (600),"Bloc H, cité des 600 Logements Makam - Laghouat",029 11 73 79,029 11 73 84,"33.795496319705656, 2.8569293262669775",POINT Z (2.85693 33.79550 0.00000)
4,04-Oum El Bouaghi,Meskiana,306,Annaba (300),"Route Radai Amar, Code 04250, Meskiana - Oum E...",032 47 18 96,032 66 22 58,"35.6334765, 7.6674872",POINT Z (7.66749 35.63348 0.00000)


In [46]:
pois_df['wilaya'] = [w[len(w)-1].strip() for w in pois_df['address'].str.split('-')]
pois_df.wilaya

0               Adrar
1               Chlef
2               Chlef
3            Laghouat
4      Oum El Bouaghi
            ...      
148              Mila
149         Aïn Defla
150    Ain Temouchent
151          Ghardaïa
152          Relizane
Name: wilaya, Length: 153, dtype: object

In [47]:
pois_df['wilaya_maj'] = [unidecode.unidecode(w) for w in pois_df.wilaya.str.upper()]
pois_df.wilaya_maj

0               ADRAR
1               CHLEF
2               CHLEF
3            LAGHOUAT
4      OUM EL BOUAGHI
            ...      
148              MILA
149         AIN DEFLA
150    AIN TEMOUCHENT
151          GHARDAIA
152          RELIZANE
Name: wilaya_maj, Length: 153, dtype: object

In [48]:
# Add Wilayas codes
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../../../dz-admin/wilayas_48.csv'
wilayas_df = pd.read_csv(wilayas_gjson)
wilayas_df = wilayas_df[['code', 'nom_maj']]
wilayas_df = wilayas_df.rename(columns={'nom_maj': 'nom_wil', 'code': 'code_wil'})
wilayas_df.head()

,code_wil,nom_wil
0,1,ADRAR
1,2,CHLEF
2,3,LAGHOUAT
3,4,OUM EL BOUAGHI
4,5,BATNA


In [49]:
# Merge with Wialyas DataFrame
merged = pois_df.merge(wilayas_df, left_on='wilaya_maj', right_on='nom_wil', how='left')

merged[merged['code_wil'].isnull()]

,Name,localite,code_agence,succursale,address,phone,fax,position,geometry,wilaya,wilaya_maj,code_wil,nom_wil


In [50]:
merged.count()

Name           153
localite       153
code_agence    153
succursale     153
address        153
phone          153
fax            153
position       153
geometry       153
wilaya         153
wilaya_maj     153
code_wil       153
nom_wil        153
dtype: int64

In [55]:
merged['lat'] = merged['geometry'].y
merged['lon'] = merged['geometry'].x

0     -0.287824
1      1.334741
2      1.341806
3      2.856929
4      7.667487
         ...   
148    6.159697
149    2.217620
150   -1.138409
151    3.697482
152    0.556412
Name: lon, Length: 153, dtype: float64

In [56]:
merged.head()

,Name,localite,code_agence,succursale,address,phone,fax,position,geometry,wilaya,wilaya_maj,code_wil,nom_wil,lat,lon
0,01-Adrar,Adrar,408,Oran (400),"N° 03, Centre Commercial ""AMRAD"" - Adrar",049 36 78 83 / 049 36 78 85,049 36 78 85,"27.870513098041783, -0.2878241762255456",POINT Z (-0.28782 27.87051 0.00000),Adrar,ADRAR,1,ADRAR,27.870513,-0.287824
1,02-Chlef,Chlef,607,Bouzaréah (600),"Rue Ibn Rochd, BP. 137 - Chlef",027 77 15 73,027 77 81 58,"36.170123, 1.334741",POINT Z (1.33474 36.17012 0.00000),Chlef,CHLEF,2,CHLEF,36.170123,1.334741
2,02-Chlef,Chlef,732,Bouzaréah (600),"Cite des fonctionnaires, Bt. 1 N°06 - Chlef",027 79 04 20,027 79 04 20,"36.169331, 1.341806",POINT Z (1.34181 36.16933 0.00000),Chlef,CHLEF,2,CHLEF,36.169331,1.341806
3,03-Laghouat,Laghouat,621,Bouzaréah (600),"Bloc H, cité des 600 Logements Makam - Laghouat",029 11 73 79,029 11 73 84,"33.795496319705656, 2.8569293262669775",POINT Z (2.85693 33.79550 0.00000),Laghouat,LAGHOUAT,3,LAGHOUAT,33.795496,2.856929
4,04-Oum El Bouaghi,Meskiana,306,Annaba (300),"Route Radai Amar, Code 04250, Meskiana - Oum E...",032 47 18 96,032 66 22 58,"35.6334765, 7.6674872",POINT Z (7.66749 35.63348 0.00000),Oum El Bouaghi,OUM EL BOUAGHI,4,OUM EL BOUAGHI,35.633477,7.667487


In [63]:
final_df = merged.drop(columns=['wilaya_maj', "wilaya", 'nom_wil', 'position', 'geometry'], axis=1)
final_df.rename_axis('num', inplace=True)
final_df.head()

,Name,localite,code_agence,succursale,address,phone,fax,code_wil,lat,lon
num,,,,,,,,,,
0,01-Adrar,Adrar,408,Oran (400),"N° 03, Centre Commercial ""AMRAD"" - Adrar",049 36 78 83 / 049 36 78 85,049 36 78 85,1,27.870513,-0.287824
1,02-Chlef,Chlef,607,Bouzaréah (600),"Rue Ibn Rochd, BP. 137 - Chlef",027 77 15 73,027 77 81 58,2,36.170123,1.334741
2,02-Chlef,Chlef,732,Bouzaréah (600),"Cite des fonctionnaires, Bt. 1 N°06 - Chlef",027 79 04 20,027 79 04 20,2,36.169331,1.341806
3,03-Laghouat,Laghouat,621,Bouzaréah (600),"Bloc H, cité des 600 Logements Makam - Laghouat",029 11 73 79,029 11 73 84,3,33.795496,2.856929
4,04-Oum El Bouaghi,Meskiana,306,Annaba (300),"Route Radai Amar, Code 04250, Meskiana - Oum E...",032 47 18 96,032 66 22 58,4,35.633477,7.667487


In [64]:
final_df.to_csv('./caar_branches_to_check.csv', header = True)